In [15]:
%load_ext autoreload
%autoreload 2

from train import *
from models import *
from tools import *

from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from keras.callbacks import *
from keras.preprocessing.image import ImageDataGenerator

from PIL import Image
from IPython.display import display

import numpy as np
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
X_train, y_train = load_data()
X_val, y_val = load_data("val")
X_test, _ = load_data("test")

y_train = to_categorical(y_train, 200)
y_val = to_categorical(y_val, 200)

Loading train data

Finished loading train data

Loading val data

Finished loading val data

Loading test data

Finished loading test data



In [17]:
labels = get_label_dict()
words = get_word_labels()

test_images = os.listdir("data/tiny-imagenet-200/test/images/")
assert len(X_test) == len(test_images)

Loading labels

Done

Loading words

Done



In [18]:
# Image preprocessing for the data
datagen = ImageDataGenerator(rotation_range=30, 
                             brightness_range=[0.3, 0.7], 
                             shear_range=0.2, 
                             zoom_range=0.2, 
                             horizontal_flip=True, 
                             width_shift_range=0.2,
                             height_shift_range=0.2, 
                             channel_shift_range=90., 
                             fill_mode="reflect", 
                             preprocessing_function=shuffle_channels, 
                             rescale=1./255)

testgen = ImageDataGenerator(rescale=1./255)

data = {"train": (X_train, y_train), 
        "val": (X_val, y_val)}

In [19]:
model = AlphaNet("v2")
model_prefix = "models/" + model.name + "/" + model.name
model_path = model_prefix + ".h5"

Model: "AlphaNet_v2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 64, 64, 32)        4736      
_________________________________________________________________
batch_normalization_8 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
spatial_dropout2d_5 (Spatial (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        51264     
_________________________________________________________________
batch_normalization_9 (Batch (None, 32, 32, 64)        

In [22]:
# Saves the best model based on monitor value between epochs
checkpoint = ModelCheckpoint(model_path, 
                             monitor="val_accuracy", 
                             verbose=1, 
                             save_best_only=True, 
                             period=1)

# Stops training early when no improvement for monitor value
early_stop = EarlyStopping(monitor="val_accuracy", 
                           min_delta=0, 
                           patience=12, 
                           verbose=1)

# Reduces learning rate when monitor value plateaus
reduce_lr = ReduceLROnPlateau(monitor="val_accuracy", 
                              factor=0.1, 
                              patience=10, 
                              verbose=1)

# Logs training data to CSV
csv_log = CSVLogger(model_prefix + ".csv", separator=',', append=False)

cb_list = [checkpoint, early_stop, reduce_lr, csv_log]

In [23]:
optim = optimizers.Adam(learning_rate=0.001)

history = train(model_path=model_path, 
                restore=False, 
                epochs=300,
                model=model, 
                optim=optim,
                datagen=datagen,
                data=data, 
                cb_list=cb_list)

Epoch 1/300
3125/3125 [==============================] - 134s 43ms/step - loss: 5.5425 - accuracy: 0.0097 - top3_accuracy: 0.0278 - top5_accuracy: 0.0446 - val_loss: 5.1318 - val_accuracy: 0.0156 - val_top3_accuracy: 0.0438 - val_top5_accuracy: 0.0697

Epoch 00001: val_accuracy improved from -inf to 0.01560, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 2/300
3125/3125 [==============================] - 134s 43ms/step - loss: 5.1092 - accuracy: 0.0232 - top3_accuracy: 0.0597 - top5_accuracy: 0.0898 - val_loss: 5.1756 - val_accuracy: 0.0387 - val_top3_accuracy: 0.0906 - val_top5_accuracy: 0.1309

Epoch 00002: val_accuracy improved from 0.01560 to 0.03870, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 3/300
3125/3125 [==============================] - 133s 42ms/step - loss: 4.8806 - accuracy: 0.0397 - top3_accuracy: 0.0961 - top5_accuracy: 0.1401 - val_loss: 4.9136 - val_accuracy: 0.0574 - val_top3_accuracy: 0.1291 - val_top5_accuracy: 0.1851

Epoch 00003: val_accurac

3125/3125 [==============================] - 128s 41ms/step - loss: 3.6429 - accuracy: 0.2025 - top3_accuracy: 0.3523 - top5_accuracy: 0.4376 - val_loss: 3.1340 - val_accuracy: 0.2228 - val_top3_accuracy: 0.3732 - val_top5_accuracy: 0.4562

Epoch 00024: val_accuracy did not improve from 0.22660
Epoch 25/300
3125/3125 [==============================] - 126s 40ms/step - loss: 3.6135 - accuracy: 0.2077 - top3_accuracy: 0.3586 - top5_accuracy: 0.4427 - val_loss: 3.3667 - val_accuracy: 0.2245 - val_top3_accuracy: 0.3817 - val_top5_accuracy: 0.4662

Epoch 00025: val_accuracy did not improve from 0.22660
Epoch 26/300
3125/3125 [==============================] - 130s 42ms/step - loss: 3.5984 - accuracy: 0.2118 - top3_accuracy: 0.3627 - top5_accuracy: 0.4453 - val_loss: 3.2617 - val_accuracy: 0.2413 - val_top3_accuracy: 0.4048 - val_top5_accuracy: 0.4933

Epoch 00026: val_accuracy improved from 0.22660 to 0.24130, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 27/300
3125/3125 [=======

3125/3125 [==============================] - 126s 40ms/step - loss: 3.2083 - accuracy: 0.2762 - top3_accuracy: 0.4452 - top5_accuracy: 0.5309 - val_loss: 3.2000 - val_accuracy: 0.3021 - val_top3_accuracy: 0.4768 - val_top5_accuracy: 0.5660

Epoch 00048: val_accuracy improved from 0.29940 to 0.30210, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 49/300
3125/3125 [==============================] - 126s 40ms/step - loss: 3.2010 - accuracy: 0.2782 - top3_accuracy: 0.4467 - top5_accuracy: 0.5338 - val_loss: 3.0972 - val_accuracy: 0.3043 - val_top3_accuracy: 0.4773 - val_top5_accuracy: 0.5636

Epoch 00049: val_accuracy improved from 0.30210 to 0.30430, saving model to models/AlphaNet_v2/AlphaNet_v2.h5
Epoch 50/300
 217/3125 [=>............................] - ETA: 1:44 - loss: 3.1586 - accuracy: 0.2815 - top3_accuracy: 0.4559 - top5_accuracy: 0.5464

KeyboardInterrupt: 

In [ ]:
show_history(history)

In [ ]:
# This loads the model and makes predictions on the test set
custom_metrics = {
    "top3_accuracy": top3_acc,
    "top5_accuracy": top5_acc
}
model = load_model(model_path, custom_objects=custom_metrics)
output = model.predict_generator(testgen.flow(X_test, shuffle=False))
results = []

for result in output:
    results.append(labels[np.argmax(result)])
    
assert len(results) == len(X_test)
print("Test images predicted:", len(results))

In [ ]:
# This writes the test set predictions to file
with open(model_prefix + " Predictions.txt", "w") as test_file:
    for i in range(len(results)):
        test_file.write(test_images[i] + " " + results[i] + "\n")
    print("Predictions saved at", test_file.name)

In [ ]:
# This shows some test set images and the model's predictions on them
display_dim = 128
for _ in range(10):
    index = np.random.randint(len(X_test))
    img = Image.fromarray(X_test[index], "RGB").resize(size=(display_dim, display_dim))
    word_label = words[results[index]]
    display(img)
    print(word_label)

In [ ]:
# This saves an HTML version of the notebook for later viewing
os.system("%notebook 'Training'")
os.system("jupyter nbconvert --to html Training.ipynb")

html_path = model_prefix + ".html"
if path.exists(html_path):
    os.remove(html_path)
os.rename("Training.html", html_path)